# Attention is All You Need

Coding a transformer from scratch


In [558]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time

### Data preprocessing pipeline


In [509]:
# Get Shakespeares work from Andrej Karpathy's website
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", url)

with open(filepath) as f:
    shakespeare_text = f.read()

In [510]:
# Print the first few characters
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [511]:
vocab = sorted(set(shakespeare_text))
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for idx, char in enumerate(vocab)}

# How many number of distinct characters has the vocabulary:
tokens_len = len(vocab)
print(f"Number of tokens in vocabulary: {tokens_len}")

# How many characters has the dataset:
text_length = len(shakespeare_text)
print(f"Total length of text dataset: {text_length}")

Number of tokens in vocabulary: 65
Total length of text dataset: 1115394


In [552]:
max_seq_len = 100

### Embedding Layer


In [512]:
embedding_dim = 10

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(tokens_len, embedding_dim))

input_array = np.random.randint(tokens_len, size=(1, 1))
model.compile("rmsprop", "sparse_categorical_crossentropy")

output_array = model.predict(input_array)
print(output_array.shape)

model.summary()
# (1, 1, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
(1, 1, 10)


Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_143 (Embedding)       │ (1, 1, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

In [513]:
def text_to_indices(text):
    return [char_to_idx[char] for char in text]


text_as_indices = text_to_indices(shakespeare_text)
print(np.array(text_as_indices))
print(len(text_to_indices(shakespeare_text)))

[18 47 56 ... 45  8  0]
1115394


In [514]:
def indices_to_text(indices):
    return "".join([idx_to_char[idx] for idx in indices])


indices_to_text([np.random.randint(65)])

's'

In [559]:
def create_training_data(text, seq_length=100, batch_size=64):
    text_as_indices = text_to_indices(text)
    total_seq = len(text_as_indices) - seq_length

    # Convert to numpy array for efficient indexing
    text_as_indices_np = np.array(text_as_indices)

    # Create input and target sequences
    input_seqs = []
    target_seqs = []
    for i in range(
        0, total_seq, seq_length // 4
    ):  # Use stride for more efficient data usage
        if i + seq_length >= total_seq:
            break
        input_seqs.append(text_as_indices_np[i : i + seq_length])
        target_seqs.append(text_as_indices_np[i + 1 : i + seq_length + 1])

    # Convert to TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((input_seqs, target_seqs))

    # Batch and shuffle
    dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)


# Create dataset
dataset = create_training_data(shakespeare_text, seq_length=max_seq_len)

### Positional Encoding


In [516]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_seq_len, embedding_dim, **kwargs):
        super().__init__(**kwargs)
        self.max_seq_len = (
            max_seq_len  # maximum sequence length that the model can handle
        )
        self.embedding_dim = embedding_dim

        # Create the positional encodings
        position = np.arange(max_seq_len)[:, np.newaxis]
        div_term = np.exp(
            np.arange(0, embedding_dim, 2) * -(np.log(10000.0) / embedding_dim)
        )
        pe = np.zeros((max_seq_len, embedding_dim))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)

        # Add batch dimension e.g. (max_seq_len,embedding_dim) -> (1,max_seq_len,embedding_dim)
        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, inputs):
        # Get the sequence length from the input shape
        seq_len = tf.shape(inputs)[1]

        # Slice the positional encoding to match the sequence length of the input
        positional_encoding = self.pe[:seq_len, :]

        # Add the positional encoding to the input embeddings
        return inputs + positional_encoding

In [517]:
embedding_dim = 10  # Embedding dimension
max_len = 50  # Maximum sequence length

# Define the model
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(tokens_len, embedding_dim),
        PositionalEncoding(max_seq_len=max_len, embedding_dim=embedding_dim),
    ]
)

input_array = np.random.randint(tokens_len, size=(1, 10))
output_array = model(input_array)

print(output_array.shape)  # Should be (1, 10, embedding_dim)

(1, 10, 10)


In [518]:
print(input_array[:3])
print(output_array[:3])

[[25 29 54 31  9 14 58 14  2 59]]
tf.Tensor(
[[[ 0.01273419  0.970314    0.0066859   0.98073745  0.0114007
    1.0440319   0.01898238  1.0143411   0.00288397  0.9877709 ]
  [ 0.8616018   0.5211885   0.1610342   0.9511266   0.04798768
    1.0276445   0.00183354  1.0187919   0.00976971  0.99095523]
  [ 0.9208757  -0.45535573  0.2810786   0.9290601   0.00203646
    0.968719   -0.03620354  0.95718205 -0.03270655  1.0164325 ]
  [ 0.18424377 -0.9802595   0.48203737  0.84386855  0.05530118
    0.9982847  -0.02168373  0.9803643  -0.03591302  0.9555277 ]
  [-0.799256   -0.6220534   0.60384     0.81232184  0.07132165
    0.99103975 -0.00262141  1.0026515  -0.01765382  1.0384865 ]
  [-0.92555386  0.2747098   0.6643983   0.69670796  0.1484447
    0.9439363   0.01071662  0.9936895   0.00352684  1.0331362 ]
  [-0.29283857  0.98933995  0.8085749   0.61624724  0.18806635
    0.9647833   0.02159707  0.95244837  0.00914372  0.9550976 ]
  [ 0.690357    0.7449498   0.8477681   0.43977898  0.19810772
    0

In [519]:
model.summary()

Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_144 (Embedding)       │ (1, 10, 10)            │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding_134         │ (1, 10, 10)            │             0 │
│ (PositionalEncoding)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

## Scaled Dot-Product Attention


In [520]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, q, k, v, mask=None):
        # dot product attention
        matmul_qk = tf.matmul(q, k, transpose_b=True)  # (bs, q_len, k_len)

        # scale dot product
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # apply mask when necessary
        if mask is not None:
            # adding very large negative values
            # so they go to zero after softmax
            scaled_attention_logits += mask * -1e9

        # apply softmax to attention weights (scores)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        # multiply by V (values)
        out = tf.matmul(attention_weights, v)

        return out, attention_weights

## Multi Head Attention Head


In [521]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        assert (
            embedding_dim % num_heads == 0
        ), "embedding_dim must be divisible by num_heads"
        self.depth = embedding_dim // num_heads  # depth per head

        # linear projection layers
        self.wq = tf.keras.layers.Dense(embedding_dim)
        self.wk = tf.keras.layers.Dense(embedding_dim)
        self.wv = tf.keras.layers.Dense(embedding_dim)

        # output projection
        self.dense = tf.keras.layers.Dense(embedding_dim)

        self.attention = ScaledDotProductAttention()

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]

        # linear projections
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        # reshaping q, k, v
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = self.attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(
            scaled_attention, (batch_size, -1, self.embedding_dim)
        )

        out = self.dense(concat_attention)

        return out, attention_weights

In [522]:
# Define dummy parameters
num_heads = 8
embedding_dim = 64
batch_size = 2
seq_length = 100  # Sequence length

# Instantiate MultiHeadAttention
mha = MultiHeadAttention(num_heads=num_heads, embedding_dim=embedding_dim)

# Create dummy input tensors
q = tf.random.uniform((batch_size, seq_length, embedding_dim))  # Queries
k = tf.random.uniform((batch_size, seq_length, embedding_dim))  # Keys
v = tf.random.uniform((batch_size, seq_length, embedding_dim))  # Values

# Run the attention layer
output, attention_weights = mha(q, k, v)

# Print output shapes
print("Query Shape: ", q.shape)
print("Key Shape: ", k.shape)
print("Value Shape: ", v.shape)
print("Output Shape: ", output.shape)
print("Attention Weights Shape: ", attention_weights.shape)

# Assertions to check correctness
assert output.shape == (
    batch_size,
    seq_length,
    embedding_dim,
), "Output shape is incorrect"
assert attention_weights.shape == (
    batch_size,
    num_heads,
    seq_length,
    seq_length,
), "Attention weights shape is incorrect"
print("✅ MultiHeadAttention test passed!")

Query Shape:  (2, 100, 64)
Key Shape:  (2, 100, 64)
Value Shape:  (2, 100, 64)
Output Shape:  (2, 100, 64)
Attention Weights Shape:  (2, 8, 100, 100)
✅ MultiHeadAttention test passed!


## Position-wise Feed-Forward Network

$$\text{FFN(x)} = \text{max}(0,~ xW_1 + b_1)W_2 + b_2$$


In [523]:
class PositionwiseFeedForward(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, hidden_dim, **kwargs):
        super().__init__(**kwargs)

        # hidden_dim (dff) - feed forward network hidden
        # layer dimension a.k.a inner layer dimensionality

        self.dense1 = tf.keras.layers.Dense(hidden_dim, activation="relu")
        self.dense2 = tf.keras.layers.Dense(embedding_dim)

    def call(self, inputs):

        x = self.dense1(inputs)

        return self.dense2(x)

## EncoderLayer


In [524]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(
        self, embedding_dim, num_heads, hidden_dim, dropout_rate=0.1, **kwargs
    ):
        super().__init__(**kwargs)

        self.mha = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = PositionwiseFeedForward(embedding_dim, hidden_dim)

        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training=False, mask=None):

        # multi-head attention
        attention_output, _ = self.mha(v=x, k=x, q=x, mask=mask)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layer_norm1(x + attention_output)

        # feed-forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layer_norm2(out1 + ffn_output)

        return out2

## DecoderLayer


In [530]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(
        self, embedding_dim, num_heads, hidden_dim, dropout_rate=0.1, **kwargs
    ):
        super().__init__(**kwargs)

        self.self_attention = MultiHeadAttention(embedding_dim, num_heads)
        self.cross_attention = MultiHeadAttention(embedding_dim, num_heads)

        self.ffn = PositionwiseFeedForward(embedding_dim, hidden_dim)

        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout3 = tf.keras.layers.Dropout(dropout_rate)

    def call(
        self, x, enc_output, training=False, look_ahead_mask=None, padding_mask=None
    ):
        # Self attention with look-ahead mask
        self_attn_output, _ = self.self_attention(q=x, v=x, k=x, mask=look_ahead_mask)
        self_attn_output = self.dropout1(self_attn_output, training=training)
        out1 = self.layer_norm1(x + self_attn_output)

        # Cross attention with encoder output
        cross_attn_output, _ = self.cross_attention(
            q=out1, v=enc_output, k=enc_output, mask=padding_mask
        )
        cross_attn_output = self.dropout2(cross_attn_output, training=training)
        out2 = self.layer_norm2(out1 + cross_attn_output)

        # Feed forward
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layer_norm3(out2 + ffn_output)

        return out3

## Encoder

The Encoder stacks multiple encoder layer to create the full encoder. It includes the Embedding layer and Positional Encoding layer as well.


In [531]:
class Encoder(tf.keras.layers.Layer):
    def __init__(
        self,
        num_layers,
        embedding_dim,
        hidden_dim,
        num_heads,
        input_vocab_size,
        max_seq_len,
        dropout_rate=0.1,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, embedding_dim)
        self.pos_encoding = PositionalEncoding(max_seq_len, embedding_dim)

        self.encoding_layers = [
            EncoderLayer(
                embedding_dim, num_heads, hidden_dim, dropout_rate=dropout_rate
            )
            for _ in range(num_layers)
        ]

        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training=False, mask=None):
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))
        x = self.pos_encoding(x)

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.encoding_layers[i](x, training=training, mask=mask)

        return x

## Decoder


In [532]:
class Decoder(tf.keras.layers.Layer):
    def __init__(
        self,
        num_layers,
        embedding_dim,
        hidden_dim,
        num_heads,
        target_vocab_size,
        max_seq_len,
        dropout_rate=0.1,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, embedding_dim)
        self.pos_encoding = PositionalEncoding(max_seq_len, embedding_dim)

        self.decoder_layers = [
            DecoderLayer(embedding_dim, num_heads, hidden_dim, dropout_rate)
            for _ in range(num_layers)
        ]

        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(
        self, x, enc_output, training=False, look_ahead_mask=None, padding_mask=None
    ):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))
        x = self.pos_encoding(x)

        x = self.dropout(x, training=training)

        # Ensure correct mask shape
        if padding_mask is not None:
            padding_mask = padding_mask[:, tf.newaxis, tf.newaxis, :]

        # decoder layers
        for i in range(self.num_layers):
            x = self.decoder_layers[i](
                x,
                enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask,
            )

        return x

## Transformer


In [ ]:
# Create look-ahead mask for decoder
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


class Transformer(tf.keras.Model):
    def __init__(
        self,
        num_layers,
        embedding_dim,
        hidden_dim,
        num_heads,
        input_vocab_size,
        target_vocab_size,
        pe_input,
        pe_target,
        dropout_rate=0.1,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.encoder = Encoder(
            num_layers,
            embedding_dim,
            hidden_dim,
            num_heads,
            input_vocab_size,
            pe_input,
            dropout_rate,
        )

        self.decoder = Decoder(
            num_layers,
            embedding_dim,
            hidden_dim,
            num_heads,
            target_vocab_size,
            pe_target,
            dropout_rate,
        )

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(
        self,
        inp,
        tar,
        enc_padding_mask=None,
        look_ahead_mask=None,
        dec_padding_mask=None,
        training=False,
    ):

        # Encoder output
        enc_output = self.encoder(x=inp, training=training, mask=enc_padding_mask)

        # Decoder output
        dec_output = self.decoder(
            x=tar,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask,
        )

        # Final output
        final_output = self.final_layer(dec_output)

        return final_output

In [543]:
# Architecture parameters
num_layers = 2
embedding_dim = 128
hidden_dim = 256
num_heads = 8
max_seq_len = 100

# Training parameters
dropout_rate = 0.1
learning_rate = 0.001

# Create model
model = Transformer(
    num_layers=num_layers,
    embedding_dim=embedding_dim,
    num_heads=num_heads,
    hidden_dim=hidden_dim,
    input_vocab_size=tokens_len,
    target_vocab_size=tokens_len,
    pe_input=max_seq_len,
    pe_target=max_seq_len,
    dropout_rate=dropout_rate,
)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Sample input
inp = tf.random.uniform((1, 10), maxval=tokens_len, dtype=tf.int32)
tar = tf.random.uniform((1, 10), maxval=tokens_len, dtype=tf.int32)

model(inp, tar)

# Print model summary
print(model.summary())

Model: "transformer_67"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_67 (Encoder)            │ ?                      │       273,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_58 (Decoder)            │ ?                      │       405,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2481 (Dense)              │ (1, 10, 65)            │         8,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 687,553 (2.62 MB)

 Trainable params: 687,553 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

None


In [555]:
# Generate text
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    # Convert start string to indices
    input_indices = text_to_indices(start_string)
    input_tensor = tf.expand_dims(input_indices, 0)

    # Empty result string
    result = start_string

    for i in range(num_generate):
        # Create look-ahead mask
        look_ahead_mask = create_look_ahead_mask(tf.shape(input_tensor)[1])

        # Call model with keyword arguments
        output = model(
            inp=input_tensor,
            tar=input_tensor,
            training=False,
            enc_padding_mask=None,
            look_ahead_mask=look_ahead_mask,
            dec_padding_mask=None,
        )

        # Select the last token from the output
        output = output[:, -1, :]  # (batch_size, vocab_size)

        # Apply temperature
        if temperature != 1.0:
            output = output / temperature

        # Sample from the output distribution
        predicted_id = tf.random.categorical(output, num_samples=1)[-1, 0].numpy()

        # Concatenate the predicted character to the output text
        result += idx_to_char[predicted_id]

        # Update the input tensor to the decoder
        input_indices.append(predicted_id)
        input_tensor = tf.expand_dims(input_indices[-max_seq_len:], 0)

    return result

In [556]:
# Generate text before training
print("\n----- Text generated before training -----\n")
start_string = "ROMEO: "
generated_text = generate_text(model, start_string, num_generate=250, temperature=1.0)
print(generated_text)


----- Text generated before training -----

ROMEO: pL&fLfezfs!R rHOb:e&bwCIaqvNQQoH!NkLzj?cgfdwIfozUwujQjYvRBHQgf
&FUplvoTjzbFAjwqz&ov;duqMBXx&aN-o'zo&R?vj&wP;vccij3dQrUd&SCWQuoC
v:jPbUOwBwqN-WAvmFv?
CBwKr&B3&J?,&?a&&&&jUoB&?sQ&vbBUP&jwaBjauvjqQtPB Scz & ;&;ccRrYBclQz
?Q
tcc&c;L&;wUgP3s&  loAVpc'fjr&


In [557]:
# Optimizer
optimizer = tf.keras.optimizers.Adam(0.001)

# Loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Use built-in metrics instead of custom class
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")


@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    # Create look-ahead mask
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar_inp)[1])

    with tf.GradientTape() as tape:
        predictions = model(
            inp=inp,
            tar=tar_inp,
            training=True,
            enc_padding_mask=None,
            look_ahead_mask=look_ahead_mask,
            dec_padding_mask=None,
        )

        # Apply mask for padding if needed
        mask = tf.math.logical_not(tf.math.equal(tar_real, 0))
        loss = loss_object(
            tar_real, predictions, sample_weight=tf.cast(mask, dtype=tf.float32)
        )

    # Get gradients and update weights
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Update metrics
    train_loss.update_state(loss)
    train_accuracy.update_state(
        tar_real, predictions, sample_weight=tf.cast(mask, dtype=tf.float32)
    )

    return loss

In [ ]:
# Training
epochs = 20
for epoch in range(epochs):
    start = time.time()

    # Reset metrics at start of each epoch
    train_loss.reset_state()  # Changed from reset_states() to reset_state()
    train_accuracy.reset_state()  # Changed from reset_states() to reset_state()

    for batch, (inp, tar) in enumerate(dataset):
        # The train_step now updates metrics internally
        loss = train_step(inp, tar)

        if batch % 50 == 0:
            print(
                f"Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} "
                f"Accuracy {train_accuracy.result():.4f}"
            )

    # Print epoch results
    print(
        f"Epoch {epoch + 1} Loss {train_loss.result():.4f} "
        f"Accuracy {train_accuracy.result():.4f}"
    )
    print(f"Time taken for 1 epoch: {time.time() - start:.2f} secs")

    # Generate text every 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"\n----- Text generated after epoch {epoch + 1} -----\n")
        start_string = "ROMEO: "
        generated_text = generate_text(
            model, start_string, num_generate=250, temperature=0.8
        )
        print(generated_text)
        print("\n")

# Save the model
model.save_weights("shakespeare_transformer/model")

# Generate final text
print("\n----- Text generated after training -----\n")
start_string = "ROMEO: "
generated_text = generate_text(model, start_string, num_generate=500, temperature=0.7)
print(generated_text)